<a href="https://colab.research.google.com/github/nariba/hello-pycuda/blob/main/SourceModule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pycuda

     |████████████████████████████████| 1.6MB 9.8MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 81kB 12.3MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621567 sha256=fcd8482db428c881ede94fc4117d98372732ad39c283ad9fe5b6848643f302f2
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=fc1c5f044d5d32c116e5f2dd41290d98c1bf5c3b425b8e26bb3b1c49f9b86ade
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


In [3]:
import math
import numpy
import pycuda.gpuarray
from pycuda.compiler import SourceModule
import pycuda.autoinit

In [4]:
module = SourceModule("""
__global__ void plus_one_kernel(int num_comp, int *y, int *x){
  int i = threadIdx.x + blockDim.x * blockIdx.x;
  if ( i < num_comp) {
    y[i] = x[i] + 1;
  }
}
""")

In [6]:
plus_one_kernel = module.get_function("plus_one_kernel")

In [8]:
num_components = numpy.int32(10)
x = numpy.arange(num_components, dtype=numpy.int32)

In [10]:
x_gpu = pycuda.gpuarray.to_gpu(x)
y_gpu = pycuda.gpuarray.zeros(num_components, dtype=numpy.int32)

In [11]:
threads_per_block = (256, 1, 1)
blocks_per_grid = (math.ceil(num_components / threads_per_block[0]), 1, 1)

In [12]:
plus_one_kernel(num_components, y_gpu, x_gpu, block=threads_per_block, grid=blocks_per_grid)

In [13]:
y_gpu.get()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int32)

In [14]:
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)